# Building a CNN to Distinguish Between Horses and Humans

In [ ]:
import urllib.request
import zipfile
import os
import requests

In [ ]:
test_dataset = "https://storage.googleapis.com/download.tensorflow.org/data/horse-or-human.zip"
validation_dataset = "https://storage.googleapis.com/download.tensorflow.org/data/validation-horse-or-human.zip"

In [ ]:
file_name = "horse-or-human.zip"
training_dir = 'horse-or-human/training/'
if not os.path.exists(training_dir) and not os.path.exists(file_name):
    urllib.request.urlretrieve(test_dataset, file_name)
    zip_ref = zipfile.ZipFile(file_name, 'r')
    zip_ref.extractall(training_dir)
    zip_ref.close()

validation_file_name = "validation-horse-or-human.zip"
validation_dir = 'horse-or-human/validation/'
if not os.path.exists(validation_dir) and not os.path.exists(validation_file_name):
    urllib.request.urlretrieve(validation_dataset, validation_file_name)
    zip_ref = zipfile.ZipFile(validation_file_name, 'r')
    zip_ref.extractall(validation_dir)
    zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import layers,Sequential,optimizers

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=(300, 300),
    batch_size=128,
    class_mode='binary'
)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1/255)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(300, 300),
    class_mode='binary'
)

In [ ]:
model = Sequential([
    layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer=optimizers.RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_generator, epochs=15, validation_data=validation_generator)

In [ ]:
import numpy as np
from keras.preprocessing import image

In [ ]:
def predict_image(value):
    img = image.load_img(value, target_size=(300, 300))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    classes = model.predict(images)
    print(classes[0])
    if classes[0]>0.5:
        print("This is a human")
    else:
        print("This is a horse")

In [ ]:
test_image = "https://static.vecteezy.com/system/resources/thumbnails/022/006/295/small_2x/horse-on-the-field-illustration-ai-generative-free-photo.jpg"

In [ ]:
img_name = "test.jpg"
img = requests.get(test_image)
with open(img_name, 'wb') as f:
    f.write(img.content)
predict_image(img_name)
